In [ ]:
import xarray as xr
import json
import pandas as pd
import xesmf as xe
import geopandas as gpd
import pandas as pd

In [ ]:
##Load data to see structure
ds = xr.open_dataset('seasonal_lst/HAQ_TEMPO_NO2_CONUS_QA75_L3_Spring2024_3PM_V3.nc')
ds

In [ ]:
# Load the shapefile with CO Census Tracts
gdf = gpd.read_file('Colorado_Census_Tract_Boundaries.geojson')
# Load state and county fips to filter data
df =pd.read_csv('state_and_county_fips_master.csv')
co_df = df[df['state']=='CO'] # Filter to only Colorado
# Create new column with County FIPS from tract fips
gdf['FIPS_new'] = gdf['FIPS'].str[:5].str.lstrip('0').astype(int)

In [ ]:
den_csa=[8031,8013,8014,8005,8059,8001,8035,8039,8093,8019,8047,8123] ##List of FIPS code in our area of interest
#Filter GDF to only include Denver CSA
filtered_gdf = gdf[gdf['FIPS_new'].isin(den_csa)]


In [ ]:
#Load spatial averager (will take a few minutes)
savg = xe.SpatialAverager(ds, filtered_gdf.geometry, geom_dim_name="tract")
savg

In [ ]:
# Convert all files to geojson with the average value for each tract
import os
files = [f for f in os.listdir('seasonal_lst') if os.path.isfile(os.path.join('seasonal_lst', f))]

for file_name in files:
    # Construct the full file path
    file_path = os.path.join('seasonal_lst', file_name)
    regs = filtered_gdf.copy()
    # Open the xarray Dataset
    ds = xr.open_dataset(file_path)
    out = savg(ds.Tropospheric_NO2)
    out = out.assign_coords(FIPS=xr.DataArray(filtered_gdf["FIPS"], dims=("tract",)))
    # Convert the Dataset to a dictionary
    regs["field_avg"] = out.values/1e15
    base = os.path.sdata_arrayitext(os.path.basename(file_name))[0]
    regs.to_file(f'../{base}.geojson', driver='GeoJSON')
    # Get the base filename without extension